# Planet API Python Client


This tutorial is an introduction to [Planet](https://www.planet.com)'s Data API using the official [Python client](https://github.com/planetlabs/planet-client-python), the `planet` module.

## Requirements

This tutorial assumes familiarity with the [Python](https://python.org) programming language throughout. Python modules used in this tutorial are:
* [IPython](https://ipython.org/) and [Jupyter](https://jupyter.org/)
* [planet](https://github.com/planetlabs/planet-client-python)
* [geojsonio](https://pypi.python.org/pypi/geojsonio)

You should also have an account on the Planet Platform and retrieve your API key from your [account page](https://www.planet.com/account/).

## Useful links 
* [Documentation](https://planetlabs.github.io/planet-client-python/index.html)
* [Planet Data API reference](https://www.planet.com/docs/reference/data-api/)

This tutorial will cover the basic operations possible with the Python client, particularly those that interact with the Data API.

The basic workflow for interacting with the Data API is:
1. search item types based on filters
1. activate assets
1. download assets

## Set up

In order to interact with the Planet API using the client, we need to import `planet.api`.

In [58]:
from planet import api

We next need to create a `ClientV1` object registered with our API key. The API key will be automatically read from the `PL_API_KEY` environment variable if it exists. If not it can be provided when creating the `api` object.

In [59]:
client = api.ClientV1()
# client = api.ClientV1(api_key="abcdef0123456789") <-- not a real key

# print client.auth.value

`ClientV1` provides basic low-level access to Planet’s API. Only one `ClientV1` should be in existence for an application. The client is thread safe and takes care to avoid API throttling and also retries any throttled requests. Any exceptional HTTP responses are handled by translation to one of the `planet.api.exceptions` classes.

We will also create a small helper function to print out JSON with proper indentation.

In [60]:
import json

def p(data):
    print(json.dumps(data, indent=2))

Let's also read in a GeoJSON geometry into a variable so we can use it during testing.

In [61]:
with open("data/san-francisco.json") as f:
    geom = json.loads(f.read())

## Searching

We can search for items that are interesting by using the `quick_search` member function. Searches, however, always require a proper request that includes a filter that selects the specific items to return as seach results.

### Filters

The Planet Python client also includes `planet.api.filters` to assist in the creation of search filters.

In [62]:
from planet.api import filters

The possible filters include `and_filter`, `date_filter`, `range_filter` and so on, mirroring the options supported by the Planet API.


In [63]:
from datetime import datetime
start_date = datetime(year=2017, month=1, day=1)

date_filter = filters.date_range('acquired', gte=start_date)
cloud_filter = filters.range_filter('cloud_cover', lte=0.1)

In [64]:
and_filter = filters.and_filter(date_filter, cloud_filter)

In [65]:
p(and_filter)

{
  "type": "AndFilter",
  "config": [
    {
      "field_name": "acquired",
      "type": "DateRangeFilter",
      "config": {
        "gte": "2017-01-01T00:00:00Z"
      }
    },
    {
      "field_name": "cloud_cover",
      "type": "RangeFilter",
      "config": {
        "lte": 0.1
      }
    }
  ]
}


In addition to the filter, a properly-constructed request also contains the list of item types that we want to select.

In [66]:
item_types = ["REOrthoTile", "PSOrthoTile"]
req = filters.build_search_request(and_filter, item_types)

In [67]:
p(req)

{
  "item_types": [
    "PSOrthoTile",
    "REOrthoTile"
  ],
  "filter": {
    "type": "AndFilter",
    "config": [
      {
        "field_name": "acquired",
        "type": "DateRangeFilter",
        "config": {
          "gte": "2017-01-01T00:00:00Z"
        }
      },
      {
        "field_name": "cloud_cover",
        "type": "RangeFilter",
        "config": {
          "lte": 0.1
        }
      }
    ]
  }
}


In [68]:
res = client.quick_search(req)

The results of `quick_search` can be handled in different ways, but most commonly the user will either iterating through the list of items (`items_iter`) or writing items to a GeoJSON file (`json_encode`).

In either case, the number of items must be specified.

In [69]:
for item in res.items_iter(4):
    print(item['id'], item['properties']['item_type'])

5013040_4256214_2021-10-19_241d PSOrthoTile
5013040_4255714_2021-10-19_241d PSOrthoTile
5013040_4252405_2021-10-19_241d PSOrthoTile
5013040_4259823_2021-10-19_241d PSOrthoTile


If the number of items requested is more than 250, the client will automatically fetch more pages of results in order to get the exact number requested.

In [70]:
with open('output/results.json','w') as f:
    res.json_encode(f,1000)

This GeoJSON file can be opened and viewed in any compatible application.

## Assets and downloads

After a search returns results, the Python client can be used to check for assets and initiate downloads.

The list of assets for an item can be retrieved with `get_assets` or `get_assets_by_id`.

In [71]:
print(item['id'])

5013040_4259823_2021-10-19_241d


In [72]:
assets = client.get_assets(item).get()

In [73]:
for asset in sorted(assets.keys()):
    print(asset)

analytic
analytic_5b
analytic_5b_xml
analytic_dn
analytic_dn_xml
analytic_xml
udm
udm2
visual
visual_xml


In [74]:
activation = client.activate(assets['analytic'])
activation.response.status_code

202

A response of 202 means that the request has been accepted and the activation will begin shortly. A 204 code indicates that the asset is already active and no further action is needed. A 401 code means the user does not have permissions to download this file.

Below, we are polling the API until the item is done activation. This may take awhile.

In [ ]:
import time

asset_activated = False

while asset_activated == False:
    
    # Get asset and its activation status
    assets = client.get_assets(item).get()
    asset = assets.get('analytic')
    asset_status = asset["status"]
    
    # If asset is already active, we are done
    if asset_status == 'active':
        asset_activated = True
        print("Asset is active and ready to download")
   
    # Still activating. Wait and check again.
    else:
        print("...Still waiting for asset activation...")
        time.sleep(3)

...Still waiting for asset activation...
...Still waiting for asset activation...
...Still waiting for asset activation...
...Still waiting for asset activation...
...Still waiting for asset activation...
...Still waiting for asset activation...
...Still waiting for asset activation...
...Still waiting for asset activation...
...Still waiting for asset activation...
...Still waiting for asset activation...
...Still waiting for asset activation...
...Still waiting for asset activation...
...Still waiting for asset activation...
...Still waiting for asset activation...
...Still waiting for asset activation...
...Still waiting for asset activation...
...Still waiting for asset activation...
...Still waiting for asset activation...
...Still waiting for asset activation...
...Still waiting for asset activation...
...Still waiting for asset activation...
...Still waiting for asset activation...
...Still waiting for asset activation...
...Still waiting for asset activation...
...Still waiting

In [ ]:
callback = api.write_to_file(directory='output/')
body = client.download(assets['analytic_xml'], callback=callback)
print(body)

## Saved Searches

The Python API client can also help in managing saved searches on the Planet Platform.

In [38]:
searches = client.get_searches()

In [39]:
for search in searches.items_iter(100):
    print(search['id'], search['name'])

6950887354d84e23889c3e308355fdcf Vancouver Island
fe44f67590ae44d6a4e5e8030de85f90 Vancouver Island


In [40]:
item_types = ["PSScene3Band"]
san_francisco_filter = filters.geom_filter(geom)
req = filters.build_search_request(san_francisco_filter, item_types, name="San Francisco")

In [41]:
p(req)

{
  "item_types": [
    "PSScene3Band"
  ],
  "filter": {
    "field_name": "geometry",
    "type": "GeometryFilter",
    "config": {
      "type": "Polygon",
      "coordinates": [
        [
          [
            -122.47455596923828,
            37.810326435534755
          ],
          [
            -122.49172210693358,
            37.795406713958236
          ],
          [
            -122.52056121826172,
            37.784282779035216
          ],
          [
            -122.51953124999999,
            37.6971326434885
          ],
          [
            -122.38941192626953,
            37.69441603823106
          ],
          [
            -122.38872528076173,
            37.705010235842614
          ],
          [
            -122.36228942871092,
            37.70935613533687
          ],
          [
            -122.34992980957031,
            37.727280276860036
          ],
          [
            -122.37773895263672,
            37.76230130281876
          ],
          [


In [42]:
res = client.create_search(req)

In [43]:
search = res.get()
print(search["id"], search["name"])

a7a648fca59245bca42d2466b289f34f San Francisco


In [44]:
res = client.saved_search(search["id"])

In [45]:
for item in res.items_iter(20):
    print(item["id"], item["properties"]["view_angle"])

20211017_180741_66_2463 5
20211017_180739_36_2463 5
20211017_183250_103b 2
20211017_183249_103b 1.9
20211015_185028_06_2416 3
20211015_185025_58_2416 3
20211015_180740_54_2206 5.1
20211015_180738_33_2206 5.1
20211015_183336_1005 1
20211015_183334_1005 1
20211015_183335_1005 1
20211015_180900_61_2432 5.1
20211014_180933_17_2223 5.1
20211014_180930_96_2223 5.1
20211014_180928_75_2223 5
20211013_180324_31_2449 3
20211013_180322_00_2449 3
20211013_180319_70_2449 3
20211012_185145_98_241c 2
20211012_182955_1039 5.1


# Statistics

The Python API client can also help report statistical summaries of the amount of data in the Planet API.

In [46]:
item_types = ["PSScene3Band"]
san_francisco_filter = filters.geom_filter(geom)
req = filters.build_search_request(san_francisco_filter, item_types, interval="year")

In [47]:
stats = client.stats(req).get()

In [48]:
p(stats)

{
  "buckets": [
    {
      "count": 3,
      "start_time": "2014-01-01T00:00:00.000000Z"
    },
    {
      "count": 60,
      "start_time": "2015-01-01T00:00:00.000000Z"
    },
    {
      "count": 200,
      "start_time": "2016-01-01T00:00:00.000000Z"
    },
    {
      "count": 1072,
      "start_time": "2017-01-01T00:00:00.000000Z"
    },
    {
      "count": 1637,
      "start_time": "2018-01-01T00:00:00.000000Z"
    },
    {
      "count": 1452,
      "start_time": "2019-01-01T00:00:00.000000Z"
    },
    {
      "count": 1434,
      "start_time": "2020-01-01T00:00:00.000000Z"
    },
    {
      "count": 1371,
      "start_time": "2021-01-01T00:00:00.000000Z"
    }
  ],
  "interval": "year",
  "utc_offset": "+0h"
}


In [49]:
assets = client.get_assets(item)

In [50]:
assets.last_modified()

In [51]:
assets.get()

{'analytic': {'_links': {'_self': 'https://api.planet.com/data/v1/assets/eyJpIjogIjIwMjExMDEyXzE4Mjk1NV8xMDM5IiwgImMiOiAiUFNTY2VuZTNCYW5kIiwgInQiOiAiYW5hbHl0aWMiLCAiY3QiOiAiaXRlbS10eXBlIn0',
   'activate': 'https://api.planet.com/data/v1/assets/eyJpIjogIjIwMjExMDEyXzE4Mjk1NV8xMDM5IiwgImMiOiAiUFNTY2VuZTNCYW5kIiwgInQiOiAiYW5hbHl0aWMiLCAiY3QiOiAiaXRlbS10eXBlIn0/activate',
   'type': 'https://api.planet.com/data/v1/asset-types/analytic'},
  '_permissions': ['download'],
  'md5_digest': None,
  'status': 'inactive',
  'type': 'analytic'},
 'analytic_dn': {'_links': {'_self': 'https://api.planet.com/data/v1/assets/eyJpIjogIjIwMjExMDEyXzE4Mjk1NV8xMDM5IiwgImMiOiAiUFNTY2VuZTNCYW5kIiwgInQiOiAiYW5hbHl0aWNfZG4iLCAiY3QiOiAiaXRlbS10eXBlIn0',
   'activate': 'https://api.planet.com/data/v1/assets/eyJpIjogIjIwMjExMDEyXzE4Mjk1NV8xMDM5IiwgImMiOiAiUFNTY2VuZTNCYW5kIiwgInQiOiAiYW5hbHl0aWNfZG4iLCAiY3QiOiAiaXRlbS10eXBlIn0/activate',
   'type': 'https://api.planet.com/data/v1/asset-types/analytic_dn'},
  '_per